In [1]:
import os
import pandas as pd
from sqlalchemy import MetaData
from sqlalchemy import create_engine, Table, Column 
from sqlalchemy import select, column
from sqlalchemy import String, Integer, Date, DateTime
from dotenv import load_dotenv


In [2]:
load_dotenv(override=True)
cxn_parameters = {
    'db_user': os.getenv('PG_DB_USER'),
    'db_password': os.getenv('PG_DB_PASSWORD'),
    'db_host': os.getenv('PG_DB_HOST'),
    'db_port': os.getenv('PG_DB_PORT'),
    'db_name': os.getenv('PG_DB_NAME')
}

db_user = cxn_parameters['db_user']
db_password = cxn_parameters['db_password']
db_user = cxn_parameters['db_user']
db_host = cxn_parameters['db_host']
db_port = cxn_parameters['db_port']
db_name = cxn_parameters['db_name']

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
metadata_obj = MetaData()

In [3]:
## reflect
tbl_bills = Table("lfg_bills", metadata_obj, schema="prod", autoload_with=engine)
tbl_bills

Table('lfg_bills', MetaData(), Column('ID', TEXT(), table=<lfg_bills>), Column('VendorName', TEXT(), table=<lfg_bills>), Column('VendorId', TEXT(), table=<lfg_bills>), Column('ReferenceNumber', TEXT(), table=<lfg_bills>), Column('Date', DATE(), table=<lfg_bills>), Column('Amount', DOUBLE_PRECISION(precision=53), table=<lfg_bills>), Column('DueDate', DATE(), table=<lfg_bills>), Column('Terms', TEXT(), table=<lfg_bills>), Column('TermsId', TEXT(), table=<lfg_bills>), Column('AccountsPayable', TEXT(), table=<lfg_bills>), Column('AccountsPayableId', TEXT(), table=<lfg_bills>), Column('Memo', TEXT(), table=<lfg_bills>), Column('IsPaid', BOOLEAN(), table=<lfg_bills>), Column('TimeModified', TIMESTAMP(), table=<lfg_bills>), Column('TimeCreated', TIMESTAMP(), table=<lfg_bills>), Column('OpenAmount', DOUBLE_PRECISION(precision=53), table=<lfg_bills>), schema='prod')

Pandas 'read_sql' method does not handle a SQLAlchemy Table object.
It will not work to simply pass a Table object into the method directly and expect it to parse.

In [ ]:
df = pd.read_sql_table(tbl_bills, con=engine)

A list of sqlalchemy Rows can be converted to a Pandas dataframes

In [4]:
with engine.connect() as conn:
    result = conn.execute(select(tbl_bills).limit(50)).fetchall()


df = pd.DataFrame(result)
df.head()

,ID,VendorName,VendorId,ReferenceNumber,Date,Amount,DueDate,Terms,TermsId,AccountsPayable,AccountsPayableId,Memo,IsPaid,TimeModified,TimeCreated,OpenAmount
0,1832F-1665592844,Jenny Lin-CS2023809R2,800004DF-1665592708,CS2023809R2,2022-09-30,300000.0,2023-09-30,12 Month,8000000B-1445897208,CS Note-Accounts Payable:Investor Capital A/P,800001BC-1447797951,capital rollover from CS2023809R1,True,2023-09-14 20:32:04,2022-10-12 17:40:44,0.0
1,182A0-1664838310,Hsiu-Chuan Chen-CS132510R9,800004D9-1664838166,CS132510R9,2022-10-02,50000.0,2023-10-02,12 Month,8000000B-1445897208,CS Note-Accounts Payable:Investor Capital A/P,800001BC-1447797951,capital rollover from CS132510R8,True,2023-09-22 19:01:44,2022-10-04 00:05:10,0.0
2,18C83-1674766452,Chuang I Lin CS2226209R1,80000505-1674766313,CS2226209R1,2022-10-02,250000.0,2023-10-02,12 Month,8000000B-1445897208,CS Note-Accounts Payable:Investor Capital A/P,800001BC-1447797951,capital rollover from CS2226209,True,2023-10-04 21:02:06,2023-01-26 20:54:12,0.0
3,18302-1665162408,Yong Mei Hu CS2226610,800004DE-1665162286,CS2226610,2022-10-07,100000.0,2023-10-07,12 Month,8000000B-1445897208,CS Note-Accounts Payable:Investor Capital A/P,800001BC-1447797951,Capital CS2226610,False,2022-10-07 18:06:48,2022-10-07 18:06:48,100000.0
4,18375-1665594954,Christopher Yale Lin-CS1610810R6,800004E1-1665594888,CS1610810R6,2022-10-07,92000.0,2023-10-07,12 Month,8000000B-1445897208,CS Note-Accounts Payable:Investor Capital A/P,800001BC-1447797951,rollover from CS1610810R5,True,2023-10-04 21:58:22,2022-10-12 18:15:54,0.0


In [5]:
upsert_row = df.copy().loc[0:2, :]
upsert_row


,ID,VendorName,VendorId,ReferenceNumber,Date,Amount,DueDate,Terms,TermsId,AccountsPayable,AccountsPayableId,Memo,IsPaid,TimeModified,TimeCreated,OpenAmount
0,1832F-1665592844,Jenny Lin-CS2023809R2,800004DF-1665592708,CS2023809R2,2022-09-30,300000.0,2023-09-30,12 Month,8000000B-1445897208,CS Note-Accounts Payable:Investor Capital A/P,800001BC-1447797951,capital rollover from CS2023809R1,True,2023-09-14 20:32:04,2022-10-12 17:40:44,0.0
1,182A0-1664838310,Hsiu-Chuan Chen-CS132510R9,800004D9-1664838166,CS132510R9,2022-10-02,50000.0,2023-10-02,12 Month,8000000B-1445897208,CS Note-Accounts Payable:Investor Capital A/P,800001BC-1447797951,capital rollover from CS132510R8,True,2023-09-22 19:01:44,2022-10-04 00:05:10,0.0
2,18C83-1674766452,Chuang I Lin CS2226209R1,80000505-1674766313,CS2226209R1,2022-10-02,250000.0,2023-10-02,12 Month,8000000B-1445897208,CS Note-Accounts Payable:Investor Capital A/P,800001BC-1447797951,capital rollover from CS2226209,True,2023-10-04 21:02:06,2023-01-26 20:54:12,0.0


In [6]:
upsert_row['ID'] = upsert_row['ID'] + 'Foo'

In [7]:
upsert_row

,ID,VendorName,VendorId,ReferenceNumber,Date,Amount,DueDate,Terms,TermsId,AccountsPayable,AccountsPayableId,Memo,IsPaid,TimeModified,TimeCreated,OpenAmount
0,1832F-1665592844Foo,Jenny Lin-CS2023809R2,800004DF-1665592708,CS2023809R2,2022-09-30,300000.0,2023-09-30,12 Month,8000000B-1445897208,CS Note-Accounts Payable:Investor Capital A/P,800001BC-1447797951,capital rollover from CS2023809R1,True,2023-09-14 20:32:04,2022-10-12 17:40:44,0.0
1,182A0-1664838310Foo,Hsiu-Chuan Chen-CS132510R9,800004D9-1664838166,CS132510R9,2022-10-02,50000.0,2023-10-02,12 Month,8000000B-1445897208,CS Note-Accounts Payable:Investor Capital A/P,800001BC-1447797951,capital rollover from CS132510R8,True,2023-09-22 19:01:44,2022-10-04 00:05:10,0.0
2,18C83-1674766452Foo,Chuang I Lin CS2226209R1,80000505-1674766313,CS2226209R1,2022-10-02,250000.0,2023-10-02,12 Month,8000000B-1445897208,CS Note-Accounts Payable:Investor Capital A/P,800001BC-1447797951,capital rollover from CS2226209,True,2023-10-04 21:02:06,2023-01-26 20:54:12,0.0


In [8]:
dict_upsert_rows = upsert_row.to_dict(orient="records")
dict_upsert_rows

[{'ID': '1832F-1665592844Foo',
  'VendorName': 'Jenny Lin-CS2023809R2',
  'VendorId': '800004DF-1665592708',
  'ReferenceNumber': 'CS2023809R2',
  'Date': datetime.date(2022, 9, 30),
  'Amount': 300000.0,
  'DueDate': datetime.date(2023, 9, 30),
  'Terms': '12 Month',
  'TermsId': '8000000B-1445897208',
  'AccountsPayable': 'CS Note-Accounts Payable:Investor Capital A/P',
  'AccountsPayableId': '800001BC-1447797951',
  'Memo': 'capital rollover from CS2023809R1',
  'IsPaid': True,
  'TimeModified': Timestamp('2023-09-14 20:32:04'),
  'TimeCreated': Timestamp('2022-10-12 17:40:44'),
  'OpenAmount': 0.0},
 {'ID': '182A0-1664838310Foo',
  'VendorName': 'Hsiu-Chuan Chen-CS132510R9',
  'VendorId': '800004D9-1664838166',
  'ReferenceNumber': 'CS132510R9',
  'Date': datetime.date(2022, 10, 2),
  'Amount': 50000.0,
  'DueDate': datetime.date(2023, 10, 2),
  'Terms': '12 Month',
  'TermsId': '8000000B-1445897208',
  'AccountsPayable': 'CS Note-Accounts Payable:Investor Capital A/P',
  'Accounts

In [9]:
with engine.begin() as conn:
    conn.execute(
        tbl_bills.insert().values(dict_upsert_rows)
    )

In [13]:
tbl_bills

Table('lfg_bills', MetaData(), Column('ID', TEXT(), table=<lfg_bills>), Column('VendorName', TEXT(), table=<lfg_bills>), Column('VendorId', TEXT(), table=<lfg_bills>), Column('ReferenceNumber', TEXT(), table=<lfg_bills>), Column('Date', DATE(), table=<lfg_bills>), Column('Amount', DOUBLE_PRECISION(precision=53), table=<lfg_bills>), Column('DueDate', DATE(), table=<lfg_bills>), Column('Terms', TEXT(), table=<lfg_bills>), Column('TermsId', TEXT(), table=<lfg_bills>), Column('AccountsPayable', TEXT(), table=<lfg_bills>), Column('AccountsPayableId', TEXT(), table=<lfg_bills>), Column('Memo', TEXT(), table=<lfg_bills>), Column('IsPaid', BOOLEAN(), table=<lfg_bills>), Column('TimeModified', TIMESTAMP(), table=<lfg_bills>), Column('TimeCreated', TIMESTAMP(), table=<lfg_bills>), Column('OpenAmount', DOUBLE_PRECISION(precision=53), table=<lfg_bills>), schema='prod')